In [14]:
import glob
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import TimeDistributed, GlobalAveragePooling2D

In [15]:
sequence_length = 10
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    frames = []
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Load and preprocess video frames
def load_video_frames(video_path, sequence_length):
    frames = frame_extract(video_path)
    num_frames = len(frames)
    if num_frames < sequence_length:
        # Pad frames if the video is shorter than the desired sequence length
        frames += [frames[-1]] * (sequence_length - num_frames)
    elif num_frames > sequence_length:
        # Randomly select a sequence of frames from the video
        start_index = random.randint(0, num_frames - sequence_length)
        frames = frames[start_index: start_index + sequence_length]

    # Resize frames to (112, 112)
    resized_frames = []
    for frame in frames:
        resized_frame = cv2.resize(frame, (224, 224)).astype(np.float32)
        resized_frames.append(resized_frame)

    # Convert frames to numpy array and preprocess
    frames_array = np.array(resized_frames, dtype=np.float32)
    frames_array = preprocess_input(frames_array)

    # Get label
    video_name = video_path.split('/')[-1]
    label = labels.loc[labels["file"] == video_name, "label"].values
    if len(label) == 0:
        return None, None
    label = label[0]
    if label == 'FAKE':
        label = 0
    elif label == 'REAL':
        label = 1

    return frames_array, label

In [18]:
from keras.models import load_model
import numpy as np
import cv2

# 저장된 모델 로드
model = load_model('best_model_1.h5')

# 테스트 데이터셋에서 영상 하나를 가져옴 (예시로 첫 번째 영상)
test_video = "./data/test/fake/aabdnomlru.mp4"
test_label = 'Fake'
# 가져온 영상을 모델에 입력하기 위해 전처리
processed_video = load_video_frames(test_video, 10)  # 예시로 전처리 함수를 사용하도록 가정

# 모델에 입력하기 위해 차원을 추가 (예시로 1개의 영상이므로 배치 차원을 추가)
processed_video = np.expand_dims(processed_video, axis=0)

# 모델로 예측 수행
predictions = model.predict(processed_video)

# 예측 결과 확인
predicted_label = np.argmax(predictions)
if predicted_label == 0:
    predicted_class = 'Fake'
else:
    predicted_class = 'Real'

# 실제 레이블 확인
if test_label == 0:
    actual_class = 'Fake'
else:
    actual_class = 'Real'

# 결과 출력
print('Predicted class:', predicted_class)
print('Actual class:', actual_class)

# 영상 시각화 (옵션)
cv2.imshow('Test Video', test_video)  # OpenCV를 사용하여 영상을 시각화할 수 있음
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'labels' is not defined

In [19]:
from keras.models import load_model

# 저장된 모델 로드
model = load_model('best_model_1.h5')

# 모델의 요약 정보 출력
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 10, 112, 11  0           []                               
                                2, 3)]                                                            
                                                                                                  
 conv3d_12 (Conv3D)             (None, 10, 112, 112  2624        ['input[0][0]']                  
                                , 32)                                                             
                                                                                                  
 conv3d_13 (Conv3D)             (None, 10, 112, 112  128         ['input[0][0]']                  
                                , 32)                                                       